In [32]:
# import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
import tables
import mplfinance as mpf
import my_afml_fncs as my_afml
from barchart_resample_bars import volume_bars

from create_bars import volume_threshold
# from AFML_Ch2_barchart import volume_bars_df
from my_afml_fncs import dropLabels, movingAverageCrossover
from research import drop_label
import seaborn as sns
from matplotlib.patches import Rectangle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

LOAD DATA

In [33]:
filename_end="2019_2024"
# volume_bars_df = pd.read_hdf("tick_data/bars/volume_bars_50000.h5", key="volume_bars")
# volume_bars_df.head()
# volume_bars_df.shape
volume_bars_df=pd.read_csv(f'tick_data/bars/volume_bars_barchart_{filename_end}.csv')
volume_bars_df['bar_number'] = range(len(volume_bars_df))
dollar_bars_df=pd.read_csv(f'tick_data/bars/dollar_bars_barchart_{filename_end}.csv')
dollar_bars_df['bar_number'] = range(len(dollar_bars_df))

Convert to datetime and set it to index

In [34]:
volume_bars_df['Date']=pd.to_datetime(volume_bars_df['Date'])
volume_bars_df.set_index('Date',inplace=True)
dollar_bars_df['Date']=pd.to_datetime(dollar_bars_df['Date'])
dollar_bars_df.set_index('Date',inplace=True)

dollar_bars_df.sort_index(inplace=True)
volume_bars_df.sort_index(inplace=True)

Create an array of daily vols

In [35]:
vol=my_afml.getDailyVol(volume_bars_df.Close)
# vol.mean()
# vol.shape
dollar_vol=my_afml.getDailyVol(dollar_bars_df.Close)

Build a cusum filter as a possible feature. Used earlier in 3.1

In [36]:
# # 3.1(a) apply CUSUM filet usings daily vols
# es_cusum_events=my_afml.getTEvents(volume_bars_df.Close,vol.mean())
# print('Volume bar CUSUM events:',es_cusum_events.shape)
# dollar_es_cusum_events=my_afml.getTEvents(dollar_bars_df.Close,dollar_vol.mean())
# print('Dollar bar CUSUM events:',dollar_es_cusum_events.shape)

Define moving average params

In [67]:
# Start of 3.4
# Define moving average windows
short_window = 20  # e.g., 50 periods
long_window = 50  # e.g., 200 periods

Create events array where the moving average switches from in to out for volume bars


In [68]:
# create events where the moving averages cross over
volume_events = my_afml.movingAverageCrossover(volume_bars_df.Close, short_window, long_window)
# events.head()
volume_events = volume_events.dropna()
# try this with a crossover signa change
volume_events['signal_switch'] = (volume_events.signal.diff()/ 2)
# drop rows with any NaN values
volume_events = volume_events.dropna()
# filter rows where signal_switch is not zero
volume_events = volume_events[volume_events['signal_switch'] != 0]


Create events array where the moving average switches from in to out for dollar bars


In [69]:
# create events where the moving averages cross over
dollar_events = my_afml.movingAverageCrossover(dollar_bars_df.Close, short_window, long_window)
# events.head()
# try this with a crossover sign change
dollar_events['signal_switch'] = dollar_events.signal.diff() / 2
# drop rows with any NaN values
dollar_events = dollar_events.dropna()
# filter rows where signal_switch is not zero
dollar_events = dollar_events[dollar_events['signal_switch'] != 0]
dollar_events.head()

,short_ma,long_ma,signal,signal_switch
Date,,,,
2018-12-31 08:04:00,2506.9750,2506.857143,1,1.0
2018-12-31 08:47:00,2506.7250,2506.787879,-1,-1.0
2018-12-31 12:01:00,2506.0375,2505.995000,1,1.0
2018-12-31 13:29:00,2507.5750,2507.875000,-1,-1.0
2018-12-31 14:35:00,2504.3250,2504.290000,1,1.0


Define vertical barriers

In [70]:
# and add vertical barrier 1 day from those events (as close to 1 day as possible?)
volume_t1=my_afml.getVb(volume_bars_df.Close, volume_events.index)
dollar_t1=my_afml.getVb(dollar_bars_df.Close,dollar_events.index)

Create meta-labels: uses the expanded getEvents to return t1 which is the timestamp the barrier was first touched and trgt which is the target that was used to generate the horizontal barrier, the expanded version accetpts a side too (and also returns it)


In [71]:
# Now create the meta-labels using the primary model (which provides the side from the crossover point), the t1 vertical barrier which is as close to 1 day as possible from the crossover. We don't label anything as primary model as that was the simple crossover. Meta-labels are used for secondary model.
# now set ptsl to be lopsided [1,2] which we can do because we now have a 'side' from the moving average crossover. Trigger has column called 'side'
ptsl=[1,2]
# target is set to be standard deviation
trgt=vol.mean()*volume_bars_df.Close
# trgt is variable but we can set a fixed minimum return
minRet=0.00003
numThread=24
# get events will find time of first touch after each event
volume_trigger_secondary_model=my_afml.getEvents(volume_bars_df.Close, volume_events.index, ptsl, trgt, minRet, numThread, volume_t1, volume_events.signal)


Running processJobs


2025-03-06 20:21:44.858640 100.0% applyPtSlOnT1 done after 0.01 minutes. Remaining 0.0 minutes.


In [73]:
#and for dollar bars
ptsl=[1,2]
# target is set to be standard deviation
dollar_trgt=dollar_vol.mean()*dollar_bars_df.Close
# trgt is variable but we can set a fixed minimum returnminRet=0.00003
numThread=24
# get events will find time of first touch after each event
dollar_trigger_secondary_model=my_afml.getEvents(dollar_bars_df.Close, dollar_events.index, ptsl, dollar_trgt, minRet, numThread, dollar_t1, dollar_events.signal)

Running processJobs


2025-03-06 20:23:19.754265 100.0% applyPtSlOnT1 done after 0.07 minutes. Remaining 0.0 minutes..


In [53]:
# volume_trigger_secondary_model.head()

In [55]:
# dollar_trigger_secondary_model.head()


Generate the bins using the triggers
The expanded get bins with a side returns 0 or 1 rather than -1, 0 or 1. So 1 is take the bet and 0 is pass (the side is already determined) and when the this secondary prediction is 1 we use the probability to size the bet!

The getBins function adds labels to the events returned by getEvents and returns the realized return of the first touched barrier and bin which is either -1,0,+1 which is the sign of the return or 0,1 if we know the side.

In [74]:
# getBins will generate the meta-label as we provide a side now in trigger
volume_bins_secondary_model=my_afml.getBins(volume_trigger_secondary_model, volume_bars_df.Close)
dollar_bins_secondary_model=my_afml.getBins(dollar_trigger_secondary_model,dollar_bars_df.Close)

In [75]:
# volume_bins_secondary_model.head()

In [76]:
# dollar_bins_secondary_model.head()

In [77]:
# add the crossover data and std to volume_bars for the secondary model
volume_bars_df[['short_ma', 'long_ma', 'signal']] = my_afml.movingAverageCrossover(volume_bars_df.Close, short_window, long_window)
volume_bars_df['std']=my_afml.getDailyVol(volume_bars_df.Close)
# volume_bars_df.head()
dollar_bars_df[['short_ma', 'long_ma', 'signal']] = my_afml.movingAverageCrossover(dollar_bars_df.Close, short_window, long_window)
dollar_bars_df['std']=my_afml.getDailyVol(dollar_bars_df.Close)
# volume_bars_df.head()

In [78]:
# now fit same data using these labels
X= volume_bars_df.loc[volume_bins_secondary_model.index, :]
y=volume_bins_secondary_model['bin']

# Prepare the data
features = X
features.reset_index(drop=True, inplace=True)
features = features.drop(columns=['Symbol'])
labels = y  # Target variable
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Train RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.5942142298670836
              precision    recall  f1-score   support

         0.0       0.62      0.56      0.59       661
         1.0       0.57      0.63      0.60       618

    accuracy                           0.59      1279
   macro avg       0.60      0.60      0.59      1279
weighted avg       0.60      0.59      0.59      1279



In [79]:
# now fit same data using these labels
dollar_X=dollar_bars_df.loc[dollar_bins_secondary_model.index,:]
dollar_y=dollar_bins_secondary_model['bin']

# Prepare the data
features = dollar_X
features.reset_index(drop=True, inplace=True)
features = features.drop(columns=['Symbol'])
labels = dollar_y  # Target variable
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Train RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.7498101334490616
              precision    recall  f1-score   support

         0.0       0.74      0.76      0.75      4591
         1.0       0.76      0.73      0.75      4626

    accuracy                           0.75      9217
   macro avg       0.75      0.75      0.75      9217
weighted avg       0.75      0.75      0.75      9217

